In [140]:
import pandas as pd
import numpy as np
import imblearn
import scipy.stats as stats
from datetime import datetime 
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
import scipy.stats as stats

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score

from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline


imblearn.__version__

'0.10.1'

In [141]:
df = pd.read_csv("merged-normalized.csv")
df = df.drop(["Date"],axis=1)
df["Time"]=pd.to_datetime(df["Time"], format='%H:%M:%S').dt.hour
#df["Time"] = stats.zscore(df["Time"])
df.head()

,Time,Race,Gender,Body_Size,With_Kids,Kids_Category,Basket_Size,Basket_colour,Attire,Shirt_Colour,...,buyDrinks,TotalSpent_RM,latitude,longitude,Num_of_Baskets,tempmax,tempmin,humidity,description,icon
0,20,malay,male,moderate,yes,young,big,red,casual,blue,...,-0.306655,-1.175289,-1.974354,0.219731,-1.24844,-0.804993,0.954235,0.51219,Cloudy skies throughout the day.,cloudy
1,20,chinese,male,thin,no,no_kids,big,green,casual,white,...,0.873818,1.620044,-1.844641,0.072282,1.25697,-0.804993,0.954235,0.51219,Cloudy skies throughout the day.,cloudy
2,20,malay,female,moderate,no,no_kids,big,blue,casual,red,...,-0.306655,-0.243511,-1.678802,1.008022,-1.24844,-0.804993,0.954235,0.51219,Cloudy skies throughout the day.,cloudy
3,21,indian,male,thin,no,no_kids,big,black,casual,black,...,-0.306655,0.455322,-1.495611,0.011978,-1.24844,-0.804993,0.954235,0.51219,Cloudy skies throughout the day.,cloudy
4,21,indian,male,moderate,no,no_kids,big,blue,casual,blue,...,-0.896892,-0.243511,-2.086209,0.252582,1.25697,-0.804993,0.954235,0.51219,Cloudy skies throughout the day.,cloudy


In [142]:
#Label encode
label_encoder = preprocessing.LabelEncoder()
for col in df:
    if df[col].dtype == "object":
        df[col]=label_encoder.fit_transform(df[col])
df.head()

,Time,Race,Gender,Body_Size,With_Kids,Kids_Category,Basket_Size,Basket_colour,Attire,Shirt_Colour,...,buyDrinks,TotalSpent_RM,latitude,longitude,Num_of_Baskets,tempmax,tempmin,humidity,description,icon
0,20,3,1,1,1,3,0,8,0,1,...,-0.306655,-1.175289,-1.974354,0.219731,-1.24844,-0.804993,0.954235,0.51219,3,0
1,20,0,1,2,0,1,0,3,0,9,...,0.873818,1.620044,-1.844641,0.072282,1.25697,-0.804993,0.954235,0.51219,3,0
2,20,3,0,1,0,1,0,1,0,8,...,-0.306655,-0.243511,-1.678802,1.008022,-1.24844,-0.804993,0.954235,0.51219,3,0
3,21,2,1,2,0,1,0,0,0,0,...,-0.306655,0.455322,-1.495611,0.011978,-1.24844,-0.804993,0.954235,0.51219,3,0
4,21,2,1,1,0,1,0,1,0,1,...,-0.896892,-0.243511,-2.086209,0.252582,1.25697,-0.804993,0.954235,0.51219,3,0


In [143]:
df =stats.zscore(df)
df.head()

,Time,Race,Gender,Body_Size,With_Kids,Kids_Category,Basket_Size,Basket_colour,Attire,Shirt_Colour,...,buyDrinks,TotalSpent_RM,latitude,longitude,Num_of_Baskets,tempmax,tempmin,humidity,description,icon
0,1.186818,1.281672,1.015622,0.007998,1.111704,1.509148,-0.855775,0.877381,-1.075186,-1.081452,...,-0.306655,-1.175289,-1.974354,0.219731,-1.24844,-0.729131,1.755352,0.320483,-1.616516,-4.855973
1,1.186818,-1.356417,1.015622,1.238486,-0.899520,-0.543765,-0.855775,-0.652490,-1.075186,1.265385,...,0.873818,1.620044,-1.844641,0.072282,1.25697,-0.729131,1.755352,0.320483,-1.616516,-4.855973
2,1.186818,1.281672,-0.984618,0.007998,-0.899520,-0.543765,-0.855775,-1.264439,-1.075186,0.972030,...,-0.306655,-0.243511,-1.678802,1.008022,-1.24844,-0.729131,1.755352,0.320483,-1.616516,-4.855973
3,1.330305,0.402309,1.015622,1.238486,-0.899520,-0.543765,-0.855775,-1.570413,-1.075186,-1.374806,...,-0.306655,0.455322,-1.495611,0.011978,-1.24844,-0.729131,1.755352,0.320483,-1.616516,-4.855973
4,1.330305,0.402309,1.015622,0.007998,-0.899520,-0.543765,-0.855775,-1.264439,-1.075186,-1.081452,...,-0.896892,-0.243511,-2.086209,0.252582,1.25697,-0.729131,1.755352,0.320483,-1.616516,-4.855973


In [144]:
X = df.drop(["TotalSpent_RM"],axis=1)
y = df["TotalSpent_RM"]
colnames = X.columns

In [145]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [171]:
lr = LinearRegression()
rfe = RFECV(lr, min_features_to_select=10, cv=3, scoring="r2")

In [172]:
rfe.fit(X,y)

RFECV(cv=3, estimator=LinearRegression(), min_features_to_select=10,
      scoring='r2')

In [173]:
def ranking(ranks, names, order=1):
    minmax = MinMaxScaler() # everything will be between 0 and 1
    ranks = minmax.fit_transform(order*np.array([ranks]).T).T[0]
    ranks = map(lambda x: round(x,2), ranks)
    return dict(zip(names, ranks))

In [174]:
rfe_score = ranking(list(map(float, rfe.ranking_)), colnames, order=-1)
rfe_score = pd.DataFrame(list(rfe_score.items()), columns=['Features', 'Score'])
rfe_score = rfe_score.sort_values("Score", ascending = False)
rfe_score

,Features,Score
0,Time,1.00
12,pants_type,1.00
2,Gender,1.00
22,Num_of_Baskets,1.00
5,Kids_Category,1.00
15,Age_Range,1.00
7,Basket_colour,1.00
1,Race,1.00
13,Wash_Item,1.00
10,shirt_type,1.00


In [175]:
for i in range(X.shape[1]):
 print('Column: %d, Selected %s, Rank: %.3f' % (i, rfe.support_[i], rfe.ranking_[i]))


Column: 0, Selected True, Rank: 1.000
Column: 1, Selected True, Rank: 1.000
Column: 2, Selected True, Rank: 1.000
Column: 3, Selected False, Rank: 9.000
Column: 4, Selected False, Rank: 14.000
Column: 5, Selected True, Rank: 1.000
Column: 6, Selected False, Rank: 16.000
Column: 7, Selected True, Rank: 1.000
Column: 8, Selected False, Rank: 17.000
Column: 9, Selected False, Rank: 13.000
Column: 10, Selected True, Rank: 1.000
Column: 11, Selected True, Rank: 1.000
Column: 12, Selected True, Rank: 1.000
Column: 13, Selected True, Rank: 1.000
Column: 14, Selected False, Rank: 3.000
Column: 15, Selected True, Rank: 1.000
Column: 16, Selected False, Rank: 11.000
Column: 17, Selected False, Rank: 7.000
Column: 18, Selected False, Rank: 10.000
Column: 19, Selected False, Rank: 2.000
Column: 20, Selected False, Rank: 4.000
Column: 21, Selected False, Rank: 8.000
Column: 22, Selected True, Rank: 1.000
Column: 23, Selected False, Rank: 15.000
Column: 24, Selected False, Rank: 18.000
Column: 25, S

In [176]:
from sklearn.model_selection import ShuffleSplit

model = LinearRegression()
pipeline = Pipeline(steps=[('s',rfe),('m',model)])
# evaluate model
cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=0)
n_scores = cross_val_score(pipeline, X, y, cv=cv, scoring='r2')
# report performance
print('R2: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

R2: -0.011 (0.007)


In [177]:
optimal_X = X.iloc[:, rfe.support_]
optimal_X

,Time,Race,Gender,Kids_Category,Basket_colour,shirt_type,Pants_Colour,pants_type,Wash_Item,Age_Range,Num_of_Baskets
0,1.186818,1.281672,1.015622,1.509148,0.877381,0.896794,-1.181502,1.090038,0.881469,-0.979143,-1.248440
1,1.186818,-1.356417,1.015622,-0.543765,-0.652490,0.896794,-0.635647,-0.917399,-1.134470,-0.635820,1.256970
2,1.186818,1.281672,-0.984618,-0.543765,-1.264439,0.896794,-1.181502,-0.917399,-1.134470,-0.807481,-1.248440
3,1.330305,0.402309,1.015622,-0.543765,-1.570413,0.896794,1.820697,1.090038,0.881469,0.994965,-1.248440
4,1.330305,0.402309,1.015622,-0.543765,-1.264439,0.896794,1.547770,-0.917399,0.881469,-0.464158,1.256970
...,...,...,...,...,...,...,...,...,...,...,...
3995,1.473792,-1.356417,1.015622,-0.543765,-0.040542,0.896794,-0.908575,1.090038,-1.134470,-0.120835,0.004265
3996,-0.104566,0.402309,1.015622,-1.570222,-0.958465,-1.115083,1.820697,-0.917399,0.881469,1.338288,0.004265
3997,1.617279,0.402309,1.015622,1.509148,-1.570413,-1.115083,-1.181502,1.090038,0.881469,1.595781,-1.248440
3998,-0.678515,-1.356417,1.015622,0.482691,-0.040542,0.896794,1.820697,1.090038,-1.134470,-1.150805,1.256970


In [178]:
# evaluate model
cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=0)
n_scores = cross_val_score(model,optimal_X, y, cv=cv, scoring='r2')
# report performance
print('R2: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

R2: -0.002 (0.007)


In [103]:
lasso = Lasso(alpha=0.9)
rfe2 = RFECV(lasso, cv=3, min_features_to_select=20, scoring="r2")

In [104]:
rfe2.fit(X,y)

RFECV(cv=3, estimator=Lasso(alpha=0.9), min_features_to_select=20, scoring='r2')

In [105]:
rfe_score = ranking(list(map(float, rfe2.ranking_)), colnames, order=-1)
rfe_score = pd.DataFrame(list(rfe_score.items()), columns=['Features', 'Score'])
rfe_score = rfe_score.sort_values("Score", ascending = False)
rfe_score

,Features,Score
14,Spectacles,1.00
15,Age_Range,1.00
26,description,1.00
25,humidity,1.00
24,tempmin,1.00
23,tempmax,1.00
22,Num_of_Baskets,1.00
21,longitude,1.00
20,latitude,1.00
19,buyDrinks,1.00


In [106]:
for i in range(X.shape[1]):
 print('Column: %d, Selected %s, Rank: %.3f' % (i, rfe2.support_[i], rfe2.ranking_[i]))


Column: 0, Selected False, Rank: 9.000
Column: 1, Selected False, Rank: 8.000
Column: 2, Selected False, Rank: 7.000
Column: 3, Selected False, Rank: 6.000
Column: 4, Selected False, Rank: 5.000
Column: 5, Selected False, Rank: 4.000
Column: 6, Selected False, Rank: 3.000
Column: 7, Selected False, Rank: 2.000
Column: 8, Selected True, Rank: 1.000
Column: 9, Selected True, Rank: 1.000
Column: 10, Selected True, Rank: 1.000
Column: 11, Selected True, Rank: 1.000
Column: 12, Selected True, Rank: 1.000
Column: 13, Selected True, Rank: 1.000
Column: 14, Selected True, Rank: 1.000
Column: 15, Selected True, Rank: 1.000
Column: 16, Selected True, Rank: 1.000
Column: 17, Selected True, Rank: 1.000
Column: 18, Selected True, Rank: 1.000
Column: 19, Selected True, Rank: 1.000
Column: 20, Selected True, Rank: 1.000
Column: 21, Selected True, Rank: 1.000
Column: 22, Selected True, Rank: 1.000
Column: 23, Selected True, Rank: 1.000
Column: 24, Selected True, Rank: 1.000
Column: 25, Selected True, 

In [107]:
model2 = Lasso(alpha=0.9)
pipeline = Pipeline(steps=[('s',rfe2),('m',model2)])
# evaluate model
cv = ShuffleSplit(n_splits=5,train_size=0.6, test_size=0.3, random_state=0)
n_scores = cross_val_score(pipeline, X, y, cv=cv, scoring='r2')
# report performance
print('R2: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

R2: -0.002 (0.002)


In [108]:
ridge = Ridge(alpha=0.9)
rfe3 = RFECV(ridge, cv=3, min_features_to_select=10)

In [109]:
rfe3.fit(X,y)

RFECV(cv=3, estimator=Ridge(alpha=0.9), min_features_to_select=10)

In [110]:
rfe_score = ranking(list(map(float, rfe3.ranking_)), colnames, order=-1)
rfe_score = pd.DataFrame(list(rfe_score.items()), columns=['Features', 'Score'])
rfe_score = rfe_score.sort_values("Score", ascending = False)
rfe_score

,Features,Score
0,Time,1.00
12,pants_type,1.00
2,Gender,1.00
22,Num_of_Baskets,1.00
5,Kids_Category,1.00
15,Age_Range,1.00
7,Basket_colour,1.00
1,Race,1.00
13,Wash_Item,1.00
10,shirt_type,1.00


In [111]:
model3 = Ridge(alpha=0.9)
pipeline = Pipeline(steps=[('s',rfe2),('m',model3)])
# evaluate model
cv = ShuffleSplit(n_splits=5,train_size=0.6, test_size=0.3, random_state=0)
n_scores = cross_val_score(pipeline, X, y, cv=cv, scoring='r2')
# report performance
print('R2: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

R2: -0.012 (0.005)


In [112]:
reg = LinearRegression().fit(X,y)
reg.score(X,y)

0.00906377781755785

In [113]:
from sklearn.preprocessing import PolynomialFeatures

In [120]:
poly = PolynomialFeatures(degree=2, include_bias=False)

In [121]:
poly_features = poly.fit_transform(X)

In [122]:
poly_model = LinearRegression()
rfe_poly = RFECV(poly_model, min_features_to_select=10, cv=3)
rfe_poly.fit(poly_features,y)

RFECV(cv=3, estimator=LinearRegression(), min_features_to_select=10)

In [125]:
rfe_score = ranking(list(map(float, rfe_poly.ranking_)), colnames, order=-1)
rfe_score = pd.DataFrame(list(rfe_score.items()), columns=['Features', 'Score'])
rfe_score = rfe_score.sort_values("Score", ascending = False)
rfe_score

,Features,Score
0,Time,1.00
12,pants_type,1.00
26,description,1.00
25,humidity,1.00
23,tempmax,1.00
22,Num_of_Baskets,1.00
20,latitude,1.00
19,buyDrinks,1.00
15,Age_Range,1.00
1,Race,1.00


In [127]:
pipeline = Pipeline(steps=[('s',rfe_poly),('m',poly_model)])
# evaluate model
cv = ShuffleSplit(n_splits=5,train_size=0.6, test_size=0.3, random_state=0)
n_scores = cross_val_score(pipeline, poly_features, y, cv=cv, scoring='r2')
# report performance
print('R2: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

R2: -0.012 (0.007)


In [132]:
poly_features.shape

(4000, 434)

MemoryError: Unable to allocate 13.8 TiB for an array with shape (4000, 472733755) and data type float64

(4000, 4494)